<font color="#6A0DAD">

This tutorial is based on https://www.tensorflow.org/text/tutorials/text_generation

<font color="#6A0DAD">

## 1 dataset

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

2023-12-31 00:24:10.788771: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-31 00:24:10.788829: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-31 00:24:10.790088: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-31 00:24:10.798020: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


<font color="#6A0DAD">

### 1.1 read dataset

In [2]:
file = open("../Zoe2.0_Markov/pattern_fix2.txt", "r")  # open our txt file, read only (which means we can't use "write()" function later)
text = file.read()

print(type(text))  # our data comes in as string
print(text[:250])  # print 0~100 elements of the string

print(f'Length of text: {len(text)} characters')

<class 'str'>
A Pattern Language is the second in a series of books which describe an entirely new attitude to architecture and planning. The books are intended to provide a complete working alternative to our present ideas about architecture, building, and planni
Length of text: 1373665 characters


In [3]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

107 unique characters


<font color="#6A0DAD">

### 1.2 perform text vectorization

<font color="#6A0DAD">

#### warm up

In [4]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

2023-12-31 00:24:16.428630: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-31 00:24:16.431018: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-31 00:24:16.469060: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [5]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)
ids_from_chars

In [6]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[63, 64, 65, 66, 67, 68, 69], [86, 87, 88]]>

In [7]:
chars_from_ids = tf.keras.layers.StringLookup( vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [8]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [9]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [10]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [11]:
text_from_ids(ids)

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'abcdefg', b'xyz'], dtype=object)>

<font color="#6A0DAD">

#### start here

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1373665,), dtype=int64, numpy=array([33,  2, 48, ..., 87, 16,  1])>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

A
 
P
a
t
t
e
r
n
 


In [16]:
seq_length = 100

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'A' b' ' b'P' b'a' b't' b't' b'e' b'r' b'n' b' ' b'L' b'a' b'n' b'g'
 b'u' b'a' b'g' b'e' b' ' b'i' b's' b' ' b't' b'h' b'e' b' ' b's' b'e'
 b'c' b'o' b'n' b'd' b' ' b'i' b'n' b' ' b'a' b' ' b's' b'e' b'r' b'i'
 b'e' b's' b' ' b'o' b'f' b' ' b'b' b'o' b'o' b'k' b's' b' ' b'w' b'h'
 b'i' b'c' b'h' b' ' b'd' b'e' b's' b'c' b'r' b'i' b'b' b'e' b' ' b'a'
 b'n' b' ' b'e' b'n' b't' b'i' b'r' b'e' b'l' b'y' b' ' b'n' b'e' b'w'
 b' ' b'a' b't' b't' b'i' b't' b'u' b'd' b'e' b' ' b't' b'o' b' ' b'a'
 b'r' b'c' b'h'], shape=(101,), dtype=string)


In [17]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'A Pattern Language is the second in a series of books which describe an entirely new attitude to arch'
b'itecture and planning. The books are intended to provide a complete working alternative to our presen'
b't ideas about architecture, building, and planning \xe2\x80\x94 an alternative which will, we hope, gradually re'
b'place current ideas and practices.\nVolume 1, The Timeless Way of Building, and Volume 2, A Pattern La'
b'nguage, are two halves of a single work. This book provides a language, for building and planning; th'


In [18]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [19]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [20]:
dataset = sequences.map(split_input_target)

In [21]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'A Pattern Language is the second in a series of books which describe an entirely new attitude to arc'
Target: b' Pattern Language is the second in a series of books which describe an entirely new attitude to arch'


In [22]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [23]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

print(vocab_size)

108


In [37]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.LSTM(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [38]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [39]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 108) # (batch_size, sequence_length, vocab_size)


2023-12-27 03:39:57.112520: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906


In [40]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  27648     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  110700    
                                                                 
Total params: 4076652 (15.55 MB)
Trainable params: 4076652 (15.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [41]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [42]:
sampled_indices

array([ 13,   3,  63,   5,  35,  46,  79,  26,  68,  79,  96,  96,  83,
        75, 101,  91,   1,  85,  38,  89,  88,  86,  58,  33,  33,  33,
        82,  12,   8,  55,   8,  19, 101,  99,  48,  89,  37,   3, 106,
        60,   5,  94,  78,  51,  33,  52,  46,  67,  55,  43,  43,  76,
        64,  28, 107,  56,  94,  90,  61,  58,  27,  56,  60,  73,  93,
        37, 101,  44,  45,  23,  26,  23,  25,  66,   5,  94,  81,  29,
        59,  46,   7,  72,  28,   3,  70,  94,  90, 103,  17,  11,  94,
        61,  63,  50,  81,   0,  32,  36,  77,  75])

In [43]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'or noxious, can be built right into homes and neighborhoods. In both cases, the crucial fact is this'

Next Char Predictions:
 b'+!a#CNq8fq\xc3\xa0\xc3\xa0um\xe2\x80\x93\xc2\xa3\nwF{zxZAAAt*&W&1\xe2\x80\x93\xce\xbbP{E!\xe2\x80\x9d]#\xc2\xb7pSATNeWKKnb:\xe2\x80\xa2X\xc2\xb7}^Z9X]k\xc2\xb1E\xe2\x80\x93LM5857d#\xc2\xb7s;[N%j:!h\xc2\xb7}\xe2\x80\x98/)\xc2\xb7^aRs[UNK]?Dom'


<font color="#6A0DAD">

## 2 train

In [44]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [45]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 108)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.6808763, shape=(), dtype=float32)


In [46]:
tf.exp(example_batch_mean_loss).numpy()

107.86455

In [47]:
model.compile(optimizer='adam', loss=loss)

In [48]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [49]:
EPOCHS = 20

In [51]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
212/212 [==============================] - 4s 12ms/step - loss: 1.0756
Epoch 2/20
212/212 [==============================] - 4s 12ms/step - loss: 1.0338
Epoch 3/20
212/212 [==============================] - 4s 12ms/step - loss: 0.9946
Epoch 4/20
212/212 [==============================] - 4s 12ms/step - loss: 0.9576
Epoch 5/20
212/212 [==============================] - 4s 12ms/step - loss: 0.9204
Epoch 6/20
212/212 [==============================] - 4s 12ms/step - loss: 0.8829
Epoch 7/20
212/212 [==============================] - 5s 13ms/step - loss: 0.8459
Epoch 8/20
212/212 [==============================] - 4s 12ms/step - loss: 0.8093
Epoch 9/20
212/212 [==============================] - 4s 12ms/step - loss: 0.7721
Epoch 10/20
212/212 [==============================] - 4s 12ms/step - loss: 0.7361
Epoch 11/20
212/212 [==============================] - 4s 11ms/step - loss: 0.7009
Epoch 12/20
212/212 [==============================] - 4s 11ms/step - loss: 0.6692
Epoch 13/20
2

<font color="#6A0DAD">

## 3 generate

In [53]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [54]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [57]:
start = time.time()
states = None
next_char = tf.constant(['A Pattern Language is'])
result = [next_char]

for n in range(3000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

A Pattern Language is the wesce based on his party grodvided the system rent in touch with the overall phape of these situations.
Now of hervowed contact with the kind of level parts of rentant and computer swimming political and human computs governing fingers — while would be possible to hill understand their shoulders and to tend to the family room in the following number of stress hinger. And it is not only place in a way which in a serious difficult for metapous there for this physical house — when the ground shell sollow the continuity.
... at each activities or not, to find himself and the fill the slab will be last high rise few ways of developing, the reasoning for the kitchen to the common land which it need.
Since the thickness of the community in place, these places are not learnd it and means to be traffic. In addition, this seems bottong to the following sequence of people all around the outskilm, semorg — When room for a day; 12 feet deep, 243 EFFICIENT STRUCTURE (206).
